# Learning Parameters with the Expectation Maximization (EM) Algorithm for Partially Observed Data

In this notebook, a simple EM algorithm is used to learn the parameters of a three node, v-structured Bayesian network such that (X -> Z <- Y) for partially observed data. 

All of the variables are drawn from a binomial distribution, that is, they are binary, {0, 1}. Each variable has a CPD table of type numpy.ndarray representing the probability distribtuion parameters - the parameters theta in the pseudo-code below.

X and Y has no parents, therefore their CPD has only two parameter estimates for P(Var=1) and P(Var=0). Z has eight parameters since it has two parents. Its CPD is a 3D numpy.ndarray such that pz=[[[p(z=0|x=0, y=0) p(z=1|x=0, y=0)], [...]], [[...], [...]]] where the indexing is such that p(z|x, y) is at pz[x, y, z]. As a result, the marginals p[x, y, :] sum to unity since p(z=0|x, y) + p(z=1|x, y) = 1. Furhermore, marginals over x and y can also be computed from the CPD of Z.

For the EM to work with missing data, the data observability model has to be Missing At Random (MAR). This conecept, and teh concept of sufficient statistics of distribtuions (here binomial) are in the centre of focus.

### Pseudo-code of EM for the v-structured Bayesian network

<pre>
<i><b>Expectation-Maximization</b></i> (
    &theta;<sup>(0)</sup> // Initial parameters
    D   // Data
  )
1    <b>for</b> each iteration t
2        M<sub>t</sub> &larr; E-step(&theta;<sup>(t)</sup>, D)
3        &theta;<sup>(t+1)</sup> &larr; M-step(M<sub>t</sub>)
4    <b>return</b> &theta;<sup>(T)</sup>

<i><b>E-step</b></i> (
    &theta;<sup>(t)</sup> // Current parameters
    D   // Data
  )
1    M<sub>t</sub> &larr; 0
2    <b>for</b> each data point d
3        <b>for</b> each node X
4            <b>for</b> each combination X=x, Pa(X)=y
5                M<sub>t</sub>[x, y] &larr; M<sub>t</sub>[x, y] + p(x, y|d, &theta;<sup>(t)</sup>)
6    <b>return</b> M<sub>t</sub>
    

<i><b>M-step</b></i> (
    M<sub>t</sub>  // Expected sufficient statistics
  )
1    <b>for</b> each node X
2        <b>for</b> each combination X=x, Pa(X)=y
3            &theta;<sup>(t+1)</sup>[x|y] &larr; M<sub>t</sub>[x, y] / M<sub>t</sub>[p]
4    <b>return</b> &#x03B8;<sup>(t+1)</sup>
</pre>

## Imports

In [1]:
import numpy as np

## Helper functions

In [2]:
def expectation_maximization(x, y, z, e_step, m_step, num_iter):
    """ Performs Expectation-Maximization algorithm and checks correctness.
    It only works for a V-structure, i.e. X -> Z <- Y
    It does not allow missing z, but both or either of x and y can be missing.
    
    Parameters
    ----------
    x, y, z : numpy.ndarray
        Input data where a None in x or y is interpreted as missing data. 
    e_step : function
        A function that takes current parameter estimates qx, qy, qz and data x, y, z 
        and outputs expected sufficient statistics Mx, My, Mz.
    m_step :function
        A function that takes current expected sufficient statistics 
        and outputs new parameter estimates qx, qy, qz.
    num_iter : int
        The number of iterations to run EM.
    
    Returns
    -------
    qx, qy, qx : numpy.ndarray
        Final parameter estimates after num_iter iterations of e_step and m_step.
    """
    n = len(x)
    qx, qy, qz = initialize_parameters()
    for i in range(num_iter):
        Mx, My, Mz = e_step(qx, qy, qz, x, y, z)
        # Assert valid statistics
        assert np.isclose(np.sum(Mx), n), f"Mx = {Mx} should sum to {n}, but is {np.sum(Mx)}"
        assert np.isclose(np.sum(My), n), f"My = {My} should sum to {n}, but is {np.sum(My)}"
        assert np.isclose(np.sum(Mz[0]), Mx[0]), \
            f"Mz[0] = {Mz[0]} should sum to Mx[0] = {Mx[0]}, but is {np.sum(Mz[0])}"
        assert np.isclose(np.sum(Mz[1]), Mx[1]), \
            f"Mz[1] = {Mz[1]} should sum to Mx[1] = {Mx[1]}, but is {np.sum(Mz[1])}"
        assert np.isclose(np.sum(Mz[:, 0]), My[0]), \
            f"Mz[:, 0] = {Mz[:, 0]} should sum to My[0] = {My[0]}, but is {np.sum(Mz[:, 0])}"
        assert np.isclose(np.sum(Mz[:, 1]), My[1]), \
            f"Mz[:, 1] = {Mz[:, 1]} should sum to My[1] = {My[1]}, but is {np.sum(Mz[:, 1])}"

        qx, qy, qz = m_step(Mx, My, Mz)
        # Assert valid parameters
        assert (qx >= 0).all(), f"qx = {qx} need to be non-negative"
        assert (qy >= 0).all(), f"qy = {qy} need to be non-negative"
        assert (qz >= 0).all(), f"qz = {qz} need to be non-negative"
        assert np.isclose(np.sum(qx), 1), f"qx = {qx} need to sum to one"
        assert np.isclose(np.sum(qy), 1), f"qy = {qy} need to sum to one"
        assert np.isclose(np.sum(qz, axis=2), 1).all(), f"Each row of qz = {qz} needs to sum to one"
    return qx, qy, qz

In [3]:
def generate_data(px, py, pz, n, *, partially_observed=False, never_coobserved=False):
    """ Generates data given table-CPDs of a V-stucture X -> Z <- Y
    It can generate complete or partially observed data
    
    Parameters
    ----------
    px, py, px : numpy.ndarray
        Parameters to generate data with.
    n : int
        Number of data points.
    partially_observed : bool
        If True, half of x and y will be missing (set to None)
    never_coobserved : bool
        If True, y is missing if and only if x is observed, so that no data points contains both x and y. 
        If False, y is missing independently of whether x is missing. 
        Has no effect if partially_observed is False.
        
    Returns
    -------
    x, y, z : numpy.ndarray
        Generated data where a None in x or y is interpreted as missing data. 
    """
    assert px.shape == (2,), f"px = {px} should have shape (2,)"
    assert py.shape == (2,), f"py = {py} should have shape (2,)"
    assert pz.shape == (2, 2, 2), f"pz = {pz} should have shape (2, 2, 2)"
    x = np.argmax(np.random.multinomial(1, px, n), axis=1)
    y = np.argmax(np.random.multinomial(1, py, n), axis=1)
    z = np.argmax([np.random.multinomial(1, p) for p in pz[x, y]], axis=1)
    if partially_observed:
        x = x.astype(object)
        y = y.astype(object)
        x[np.unique(np.random.choice(n, int(n/2)))] = None
        if never_coobserved:
            y[np.not_equal(x, None)] = None
        else:
            y[np.unique(np.random.choice(n, int(n/2)))] = None
    return x, y, z

In [4]:
def initialize_parameters(random=False):
    """ Initializes parameters for the EM algorithm
    
    Parameters
    ----------
    random : bool
        If True, the parameters are set to random values (in range [0, 1] that sum to 1).
        If False, all probabilities are 0.5 (binary variables).
    
    Returns
    -------
    qx, qy, qx : numpy.ndarray
        Initial parameters.
    """
    if random:
        qx = np.random.rand(2)
        qy = np.random.rand(2)
        qz = np.random.rand(2, 2, 2) 
    else:
        qx = np.ones(2)
        qy = np.ones(2)
        qz = np.ones((2,2,2))
    qx = qx / np.sum(qx)
    qy = qy / np.sum(qy)
    qz = qz / np.sum(qz, axis=2, keepdims=1)
    return qx, qy, qz

In [5]:
def print_tables(px, py, pz):
    """ Prints probability tables in a nice way. 
    
    Parameters
    ----------
    px, py, pz : numpy.ndarray
        Parameters, true or learnt.
    
    Returns
    -------
    None
    """
    assert px.shape == (2,), f"px = {px} should have shape (2,)"
    assert py.shape == (2,), f"py = {py} should have shape (2,)"
    assert pz.shape == (2, 2, 2), f"pz = {pz} should have shape (2, 2, 2)"
    print(f"p(x) = {px}")
    print(f"p(y) = {py}")
    print(f"p(z|x=0, y=0) = {pz[0, 0]}")
    print(f"p(z|x=0, y=1) = {pz[0, 1]}")
    print(f"p(z|x=1, y=0) = {pz[1, 0]}")
    print(f"p(z|x=1, y=1) = {pz[1, 1]}")


def print_marginals(px, py, pz):
    """ Prints marginal probabilities of z given x or y, i.e. one variable is summed out.
    
    Parameters
    ----------
    px, py, pz : numpy.ndarray
        Parameters, true or learnt.
    Returns
    -------
    None
    """
    assert px.shape == (2,), f"px = {px} should have shape (2,)"
    assert py.shape == (2,), f"py = {py} should have shape (2,)"
    assert pz.shape == (2, 2, 2), f"pz = {pz} should have shape (2, 2, 2)"
    print(f"p(z|x=0) = {pz[0, 0] * py[0] + pz[0, 1] * py[1]}")
    print(f"p(z|x=1) = {pz[1, 0] * py[0] + pz[1, 1] * py[1]}")
    print(f"p(z|y=0) = {pz[0, 0] * px[0] + pz[1, 0] * px[1]}")
    print(f"p(z|y=1) = {pz[0, 1] * px[0] + pz[1, 1] * px[1]}")

## EM algoritm

### Expectation step

In [6]:
def e_step(qx, qy, qz, xs, ys, zs):
    """ Expectation step of EM.
    Computes p(X|x, y, z), p(Y|x, y, z), and p(Z, X, Y|x, y, z), 
    normalizes them so each sums to 1, and adds to Mx, My, and Mz, respectively. 
    
    Parameters
    ----------
    qx, qy, qx : numpy.ndarray
        ML parameter estimates of EM. Output of Maximization step in EM.
    xs, ys, zs : numpy.ndarray
        Generated data where a None in x or y is interpreted as missing data.
    
    Returns
    -------
    Mx, My, Mz : numpy.ndarray
        Sufficient statistics of the distribtuions of x,y, and z. Output of Expectation step in EM.
    """
    Mx = np.zeros(2)
    My = np.zeros(2)
    Mz = np.zeros((2, 2, 2))
    for x, y, z in zip(xs, ys, zs):
        Q = np.zeros((2,2))
        
        if x is not None and y is not None:
            Mx[x] += 1
            My[y] += 1
            Mz[x,y,z] += 1
        else:
            if x is None and y is not None:
                Q[0,0] = qx[0] * (1-y) * qz[0,0,z]
                Q[0,1] = qx[0] * y * qz[0,1,z]
                Q[1,0] = qx[1] * (1-y) * qz[1,0,z]
                Q[1,1] = qx[1] * y * qz[1,1,z]
            elif x is not None and y is None:
                Q[0,0] = (1-x) * qy[0] * qz[0,0,z]
                Q[0,1] = (1-x) * qy[1] * qz[0,1,z]
                Q[1,0] = x * qy[0] * qz[1,0,z]
                Q[1,1] = x * qy[1] * qz[1,1,z]
            else:
                Q[0,0] = qx[0] * qy[0] * qz[0,0,z]
                Q[0,1] = qx[0] * qy[1] * qz[0,1,z]
                Q[1,0] = qx[1] * qy[0] * qz[1,0,z]
                Q[1,1] = qx[1] * qy[1] * qz[1,1,z]
                
            Q /= np.sum(Q)
            assert np.isclose(np.sum(Q), 1.0), f"Q:{Q}, x:{x}, y:{y}, z:{z}"
            
            if x is None:
                Mx += np.sum(Q, axis=1)
            else:
                Mx[x] += 1
            if y is None:
                My += np.sum(Q, axis=0)
            else:
                My[y] += 1
            
            Mz[:,:,z] += Q
            
    return Mx, My, Mz

### Maximization step

In [7]:
def m_step(Mx, My, Mz):
    """ Maximization step of EM. Converts the sufficient statistics to ML parameter estimates.
    
    Parameters
    ----------
    Mx, My, Mz : numpy.ndarray
        Sufficient statistics of the distribtuions of x,y, and z. Output of Expectation step in EM.
    
    Returns
    -------
    qx, qy, qx : numpy.ndarray
        ML parameter estimates of EM. Output of Maximization step in EM.
    """
    qx = np.zeros(2)
    qy = np.zeros(2)
    qz = np.zeros((2,2,2))
    
    qx = Mx / np.sum(Mx)
    qy = My / np.sum(My)
    
    qz[0,0,0] = Mz[0,0,0] / np.sum(Mz[0,0,:])
    qz[0,0,1] = Mz[0,0,1] / np.sum(Mz[0,0,:])
    
    qz[0,1,0] = Mz[0,1,0] / np.sum(Mz[0,1,:])
    qz[0,1,1] = Mz[0,1,1] / np.sum(Mz[0,1,:])
    
    qz[1,0,0] = Mz[1,0,0] / np.sum(Mz[1,0,:])
    qz[1,0,1] = Mz[1,0,1] / np.sum(Mz[1,0,:])
    
    qz[1,1,0] = Mz[1,1,0] / np.sum(Mz[1,1,:])
    qz[1,1,1] = Mz[1,1,1] / np.sum(Mz[1,1,:])
    
    return qx, qy, qz

## EM with no missing data

Having implemented the E-step and the M-step for complete data the results shown below were obtained with the numpy random seed set to 1337

In [8]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 500
x, y, z = generate_data(px, py, pz, n_data)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.568 0.432]
p(y) = [0.294 0.706]
p(z|x=0, y=0) = [0.24137931 0.75862069]
p(z|x=0, y=1) = [0.63451777 0.36548223]
p(z|x=1, y=0) = [0.88333333 0.11666667]
p(z|x=1, y=1) = [0.1025641 0.8974359]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


As shown in the print-out above, in the case of complete data, the EM algorithm accurately estimates the parameters of the Bayesian network.

When the random seed is set to 2018, the results shown below were obtained.

In [9]:
np.random.seed(2018)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 500
x, y, z = generate_data(px, py, pz, n_data)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.586 0.414]
p(y) = [0.316 0.684]
p(z|x=0, y=0) = [0.20212766 0.79787234]
p(z|x=0, y=1) = [0.70351759 0.29648241]
p(z|x=1, y=0) = [0.890625 0.109375]
p(z|x=1, y=1) = [0.09090909 0.90909091]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


Analogously to the previous random seed, in the case of the random seed of 2018, the network parameters are accurately estimated with the EM algorithm.

## EM with missing data

When estimating the parameters of the network with the EM algorithm for partially observed data, the results shown below were obtained with the random seed set to 1337. 

In [10]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])
n_data = 500
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.58130798 0.41869202]
p(y) = [0.28239202 0.71760798]
p(z|x=0, y=0) = [0.28220124 0.71779876]
p(z|x=0, y=1) = [0.60052118 0.39947882]
p(z|x=1, y=0) = [0.92109464 0.07890536]
p(z|x=1, y=1) = [0.09122205 0.90877795]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


When the seed was set to 2018, the results shown below were obtained.

In [11]:
np.random.seed(2018)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])
n_data = 500
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.61579605 0.38420395]
p(y) = [0.31324231 0.68675769]
p(z|x=0, y=0) = [0.24857963 0.75142037]
p(z|x=0, y=1) = [0.69335717 0.30664283]
p(z|x=1, y=0) = [0.87966812 0.12033188]
p(z|x=1, y=1) = [0.06643502 0.93356498]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


In the case of the random seed of 2018, the network parameters are again accurately estimated with the EM algorithm for partially observed data.

## EM with missing data with modified generating distributions and fewer instances in data set

Firstly, experiments were run with modified generating distributions (px, py, pz), then experiments were run on data with only a few instances.

In the first experiment, the generating distributions were modified to have different parameters. The results shown below were obtained from learning the parameters of the Bayesian network with the EM algorithm for partially observed data, with the random seed set to 1337.

In [12]:
np.random.seed(1337)
px = np.array([0.5, 0.5])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.4, 0.6]], [[0.85, 0.15], [0.1, 0.9]]])
n_data = 500
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.51679318 0.48320682]
p(y) = [0.28320376 0.71679624]
p(z|x=0, y=0) = [0.16650611 0.83349389]
p(z|x=0, y=1) = [0.39762683 0.60237317]
p(z|x=1, y=0) = [0.78143991 0.21856009]
p(z|x=1, y=1) = [0.17100803 0.82899197]

True parameters
---------------
p(x) = [0.5 0.5]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.4 0.6]
p(z|x=1, y=0) = [0.85 0.15]
p(z|x=1, y=1) = [0.1 0.9]


As shown in the print-out results, the parameters learned by the EM algorithm are accurate.

In the second experiment, the generating distributions were modified with the aim to evaluate the performance of the EM algorithm when some probabilities are set to zero. With the random seed set to 1337, the results shown below were obtained.

In [13]:
np.random.seed(1337)
px = np.array([0.0, 1.0])
py = np.array([0.5, 0.5])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.0, 1.0]]])
n_data = 500
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [4.79278354e-05 9.99952072e-01]
p(y) = [0.52123828 0.47876172]
p(z|x=0, y=0) = [0.9387989 0.0612011]
p(z|x=0, y=1) = [6.49002116e-04 9.99350998e-01]
p(z|x=1, y=0) = [0.91702921 0.08297079]
p(z|x=1, y=1) = [1.81026233e-05 9.99981897e-01]

True parameters
---------------
p(x) = [0. 1.]
p(y) = [0.5 0.5]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0. 1.]


As shown in the results above, the EM algorithm did not succeed in accurately learning all of the parameters. The generating distributions px and py for node X and Y in the network are accurately estimated. However, the estimations for the generating distribution pz for node Z in the network have shortcomings. Firstly, the learned distributions p(z|x=1, y=0) and p(z|x=1, y=1) are accurate when compared to their true counterparts. Nevertheless, the learned distributions p(z|x=0, y=0) and p(z|x=0, y=1) are rather inaccurate of their true counterparts.

In the third experiment, the generating distribtuions were reset to their original values and the number of instances in the partially observed data set were reduced to n_data = 100. With the the random seed set to 1337, the results shown below were obtained.

In [14]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 100
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.64843219 0.35156781]
p(y) = [0.25703223 0.74296777]
p(z|x=0, y=0) = [0.23808872 0.76191128]
p(z|x=0, y=1) = [0.49356376 0.50643624]
p(z|x=1, y=0) = [0.79389173 0.20610827]
p(z|x=1, y=1) = [0.19638459 0.80361541]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


As shown above, reducing the number of instances in the partially observed data set from the original 500 to 100 had a slight detrimental effect on the accuracy of the parameters learned by the EM algorithm. The modest degradation of the accuracy manifests itself in all of the learned parameters, the most inaccurate of which is the generating distribution p(z|x=0, y=1). Nevertheless, the parameters estimations are still generally accurate.

In the fourth and last experiment, the number of instances in the data set was further reduced from 100 to 10. With the random seed set to 1337, the results shown below were obtained.

In [15]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 10
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=False)
n_iter = 50
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.81798999 0.18201001]
p(y) = [0.22019805 0.77980195]
p(z|x=0, y=0) = [0. 1.]
p(z|x=0, y=1) = [0.47519945 0.52480055]
p(z|x=1, y=0) = [0. 1.]
p(z|x=1, y=1) = [1.28623669e-10 1.00000000e+00]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]


As shown in the results above, the EM algorithm did not succeed at learning the parameters with the exception of the generating distribution p(y), which is approximately close to its true counterpart.

Generally, the EM algorithm accurately estimates the parameters of the generating distributions of the Bayesian network when the parameters of the true generating distributions are modified. However, reducing the number of instances in the partially observed data set deteriorates the accuracy of the estimates. For a mere 10 data points, the algorithm fails to learn accurate parameter estimates. 

## EM with missing data that are never co-observed

In this section, the variables x and y (the parents of z) are never co-observed such that y is missing if and only if x is not missing, i.e. all data points are either (None, y, z) or (x, None, z).

Using the original generating distributions the results shown below were obtained for learning the parameters of the Bayesian network with the EM algorithm for partially observed, and never co-observed data, with the random seed set to 1337.

In [16]:
np.random.seed(1337)
px = np.array([0.6, 0.4])
py = np.array([0.3, 0.7])
pz = np.array([[[0.2, 0.8], [0.7, 0.3]], [[0.9, 0.1], [0.1, 0.9]]])  # p(z|x, y) = pz[x, y, z]
n_data = 500
x, y, z = generate_data(px, py, pz, n_data, partially_observed=True, never_coobserved=True)
n_iter = 10
qx, qy, qz = expectation_maximization(x, y, z, e_step, m_step, n_iter)

print("Learnt parameters")
print("-----------------")
print_tables(qx, qy, qz)
print()
print("True parameters")
print("---------------")
print_tables(px, py, pz)

print()
print("Learnt marginals")
print("----------------")
print_marginals(qx, qy, qz)
print()
print("True marginals")
print("--------------")
print_marginals(px, py, pz)

Learnt parameters
-----------------
p(x) = [0.57023911 0.42976089]
p(y) = [0.28274479 0.71725521]
p(z|x=0, y=0) = [0.52164892 0.47835108]
p(z|x=0, y=1) = [0.51070207 0.48929793]
p(z|x=1, y=0) = [0.28882444 0.71117556]
p(z|x=1, y=1) = [0.33063173 0.66936827]

True parameters
---------------
p(x) = [0.6 0.4]
p(y) = [0.3 0.7]
p(z|x=0, y=0) = [0.2 0.8]
p(z|x=0, y=1) = [0.7 0.3]
p(z|x=1, y=0) = [0.9 0.1]
p(z|x=1, y=1) = [0.1 0.9]

Learnt marginals
----------------
p(z|x=0) = [0.51379724 0.48620276]
p(z|x=1) = [0.31881093 0.68118907]
p(z|y=0) = [0.42159006 0.57840994]
p(z|y=1) = [0.43331488 0.56668512]

True marginals
--------------
p(z|x=0) = [0.55 0.45]
p(z|x=1) = [0.34 0.66]
p(z|y=0) = [0.48 0.52]
p(z|y=1) = [0.46 0.54]


As shown in the results above, the EM algorithm did not succeed at estimating the generating distribution parameters with the exception of p(x) and p(y), which are approximately close to their true counterparts. Since the nodes x and y are the parents of z, always missing either one of the parent values significantly worsens the estimation of the conditional generating distribution p(z|x, y) of node z. As a result, the parameter estimates of the distribution $p(z|x, y)$ are nowhere near their true values. The notably worse parameter estimates are due to the fact the data observability model is not Missing At Random (MAR) anymore since the values of the observation variables of x and y are not independent of the missing data points. What is more, actually, a non-missing x causes y to be missing and a missing x causes y to be non-missing. For the EM algorithm to work, the data observability model has to be MAR, or be converted to MAR via adding more nodes.

## TO-DO

1. Add code to visualize the expected complete data log-likelihood and how it is maximized until a local maximum during the iterations of the EM.
2. Add more nodes to convert the "never co-observed x and y" case to MAR. Extend EM algorithm to accomodate the new node/nodes.

## References

1. DD2420 Probabilistic Graphical Models Course Lecture Notes and Laboratory Guide at the KTH Royal Institute of Technology (some of the code was based on notes and lab documents)
2. Probabilistic Graphical Models: Principles and Techniques by Daphne Koller and Nir Friedman (some of the ideas for the code were based on their outstanding book)